In [1]:
import torch
from torchvision import models

pretrained_model = models.resnet50(weights = 'DEFAULT', progress = True)

In [2]:
n = 100 #let n = the number of types of plants

In [3]:
# changes fully connected/classifier layer to new layer for us to train
pretrained_model.fc = torch.nn.Linear(2048, 2)

In [7]:
# Currently no in use
import torch.nn as nn
# build custom softmax module
class Softmax(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        self.linear = nn.Linear(n_inputs, n_outputs)
 
    def forward(self, x):
        pred = self.linear(x)
        return pred

In [8]:
# Currently no in use
#add softmax to model
class MyModel(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.last_layer = Softmax(1000, n) # add how many nodes as input and output

    def forward(self, x):
        return self.last_layer(self.pretrained_model(x))

model = MyModel(pretrained_model)

In [11]:
#freeze model except fc layer
for param in pretrained_model.parameters():
    param.requires_grad = False

for param in pretrained_model.fc.parameters():
    param.requires_grad = True

In [12]:
criterion = torch.nn.CrossEntropyLoss() #could write this out ourselves
# need to find an optimizer or make onefor custom softmax function
optimizer = torch.optim.SGD(pretrained_model.fc.parameters(), lr=0.001, momentum=0.9)

In [13]:
# should probably make some changes here to get what we want

import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(224), # change to waht data should be
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print some statistics
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


NameError: name 'device' is not defined